Задание №1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ratings13.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [9]:
#количество оценок
ratings.shape

(100004, 4)

In [10]:
#количество уникальных пользователей
len(ratings['userId'].unique())

671

In [13]:
#пользователи, которые выставили более 100 оценок
score = ratings.groupby('userId').count().reset_index()
score.rename(columns = {'rating':'count_ratings'}, inplace = True)
score = score[score.count_ratings > 100]
score = score.sort_values(by = 'count_ratings', ascending = False)
score = score[['userId', 'count_ratings']]
score.head()

,userId,count_ratings
546,547,2391
563,564,1868
623,624,1735
14,15,1700
72,73,1610


In [14]:
#количество пользователей, которые выставили более 100 оценок
score.shape

(258, 2)

In [15]:
#объединение
joined = score.merge(ratings, how='left')
joined.head()

,userId,count_ratings,movieId,rating,timestamp
0,547,2391,1,3.5,1053173138
1,547,2391,6,2.5,1149462857
2,547,2391,7,2.0,1021098817
3,547,2391,11,3.0,1418149949
4,547,2391,14,3.5,1053171195


In [16]:
#среднее время жизни пользователей
joined = joined.groupby('userId').agg([min,max])
joined.head()

count_ratings       movieId         rating        timestamp            
                 min   max     min     max    min  max         min         max
userId                                                                        
4                204   204      10    4006    1.0  5.0   949778714   949982274
8                116   116      32   44004    0.5  5.0  1154389340  1154474527
15              1700  1700       1  161155    0.5  5.0   997937239  1469330735
17               363   363       6   34437    0.5  5.0  1127468587  1127476640
19               423   423       1    4970    1.0  5.0   855190091   855195373

In [18]:
joined['diff'] = joined['timestamp']['max'] - joined['timestamp']['min']
var_ = joined['diff'].mean()
var_ = round(var_/24/60/60,2)
print(f'среднее время жизни пользователей, которые выставили более 100 = {var_}(дней)')

среднее время жизни пользователей, которые выставили более 100 = 463.89(дней)


Задание №2. Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки,
- auto - автомобильные перевозки,
- air - воздушные перевозки,
- client_base - адреса клиентов.

Необходимо сформировать две таблицы:
1. Таблицу с тремя типами выручки для каждого client_id без указания адреса клиента.
2. Аналогичную таблицу по типам выручки с указанием адреса клиента.

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [24]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd
#rzd.sum()

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [20]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto
#auto.sum()

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [21]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air
#air.sum()

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [29]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [30]:
data_rzd_auto = rzd.merge(auto, how='outer', on='client_id')
data_rzd_auto
#data_rzd_auto.sum()

,client_id,rzd_revenue,auto_revenue
0,111,1093.0,NaN
1,112,2810.0,NaN
2,113,10283.0,57483.0
3,114,5774.0,83.0
4,115,981.0,912.0
5,116,NaN,4834.0
6,117,NaN,98.0


In [31]:
data_rzd_auto_air = data_rzd_auto.merge(air, how='outer', on='client_id')
data_rzd_auto_air
#data_rzd_auto_air.sum()

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [32]:
data_address = data_rzd_auto_air.merge(client_base, how='outer', on='client_id')
data_address
#data_address.sum()

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
